In [7]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

In [8]:
def interference(loc1, loc2, DI):
    dist = math.sqrt((loc1[0]-loc2[0])**2+(loc1[1]-loc2[1])**2)
    return dist < DI

def cal_utility(an,aJn):
    sn = 0
    for ak in aJn:
        f = 1 if an==ak else 0
        sn = sn+f
    gn = csma_p*(1-csma_p)**sn
    return gn, sn

In [10]:
# parameters defination
max_step = 1000
beta_division=100
csma_p = 0.5
area_w = 1000
area_h = 1000
DI_jammer = 250
DI_usr = 250
usr_num = 20
channel_num = 5

jammer_loc = [(300,200), (700,300), (250,600), (750,700)]
jammer_channel = [random.randint(0, channel_num-1) for i in range(len(jammer_loc))]

usr_loc = [(random.randint(0, area_w-1),random.randint(0, area_h-1)) 
        for i in range(usr_num)]

usr_channel_set = []
for i, loc1 in enumerate(usr_loc):
    channel_set = [l for l in range(channel_num)]
    for j,loc2 in enumerate(jammer_loc):
        if interference(loc1,loc2,DI_jammer):
            channel_set.remove(jammer_channel[j])
    usr_channel_set.append(channel_set)

usr_near_set = []
for i, loc1 in enumerate(usr_loc):
    near_set = []
    for j, loc2 in enumerate(usr_loc):
        if i==j: continue
        if interference(loc1,loc2,DI_usr):
            near_set.append(j)
    usr_near_set.append(near_set)
    
usr_channel = []
usr_metric1 = []
usr_metric2 = []
    
usr_utility1 = []
usr_utility2 = []
usr_strategy = []
for i, channel_set in enumerate(usr_channel_set):
    channel_num = len(channel_set)
    sgm = 1/channel_num
    usr_strategy.append([sgm for i in range(channel_num)])
    usr_utility1.append([0 for i in range(channel_num)])
    usr_utility2.append([0 for i in range(channel_num)]) 

In [11]:
def visulize(colors):
    fig = plt.figure(figsize=(6,8), dpi=100, tight_layout=True)
    ax = fig.add_subplot(111)
    
    for i, loc in enumerate(jammer_loc):
        xy = (loc[0], loc[1])
        cir1 = Circle(xy, radius=DI_jammer, fill=False, 
                      edgecolor='k', alpha=0.1, linestyle='--')
        ax.add_patch(cir1)
        
    for n, near_set in enumerate(usr_near_set):
        for k in near_set:
            x = (usr_loc[n][0], usr_loc[k][0])
            y = (usr_loc[n][1], usr_loc[k][1])
            ax.plot(x,y,'k',alpha=0.1)
            
    x = [jammer_loc[i][0] for i in range(len(jammer_loc))]
    y = [jammer_loc[i][1] for i in range(len(jammer_loc))]
    plt.scatter(x, y, marker='X',c=jammer_channel, s=200, cmap='rainbow')
    
    x = [usr_loc[i][0] for i in range(usr_num)]
    y = [usr_loc[i][1] for i in range(usr_num)]
    plt.scatter(x, y, c=colors, s=50, edgecolor='k', cmap='rainbow')
    
    plt.colorbar(ticks=np.linspace(0, channel_num, channel_num+1),
                orientation='horizontal'); # show color scale

In [12]:
#SAP begins
for kk in range(max_step):
    actions = [np.random.choice(usr_channel_set[i],1,usr_strategy[i])[0] 
               for i in range(usr_num)]
    usr_channel.append(actions)
    usr_gn = []
    usr_sn = []
    for i in range(usr_num):
        gn = csma_p
        sn = 0
        for _, j in enumerate(usr_near_set[i]):
            f = 1 if actions[i]==actions[j] else 0
            sn = sn+f
        gn = gn*(1-csma_p)**sn
        usr_gn.append(gn)
        usr_sn.append(sn)
    usr_metric1.append(usr_gn)
    usr_metric2.append(usr_sn)
    usr_n = random.randint(0,usr_num-1)
    An = usr_channel_set[usr_n]
    for i, a in enumerate(An):
        an = a
        Jn = usr_near_set[usr_n]
        aJn = [actions[l] for l in Jn]
        gn, sn = cal_utility(an,aJn)
        gJn = []
        for k in Jn:
            ak = actions[k]
            Jk = usr_near_set[k]
            aJk = []
            for l in Jk:
                if l==usr_n:
                    aJk.append(a)
                    continue
                aJk.append(actions[l])
            gk, sk = cal_utility(ak,aJk)
            gJn.append(gk) 
        usr_utility1[usr_n][i] = gn+sum(gJn)
        usr_utility2[usr_n][i] = -sn
    usr_utility = usr_utility1
    #usr_utility = usr_utility2
    beta = kk/beta_division        
    sum_uti = sum([math.exp(beta*usr_utility[usr_n][i]) 
                   for i,a in enumerate(An)])
    for i, a in enumerate(An):
        usr_strategy[usr_n][i]=(math.exp(beta*usr_utility[usr_n][i])/sum_uti)

In [ ]:
def update_points(num):
    if num%5==0:
        point_ani.set_marker("*")
        point_ani.set_markersize(12)
    else:
        point_ani.set_marker("o")
        point_ani.set_markersize(8)

    point_ani.set_data(x[num], y[num])    
    text_pt.set_text("x=%.3f, y=%.3f"%(x[num], y[num]))
    return point_ani,text_pt,

x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x)

fig = plt.figure(tight_layout=True)
plt.plot(x,y)
point_ani, = plt.plot(x[0], y[0], "ro")
plt.grid(ls="--")
text_pt = plt.text(4, 0.8, '', fontsize=16)

ani = animation.FuncAnimation(fig, update_points, np.arange(0, 100), interval=100, blit=True)

# ani.save('sin_test3.gif', writer='imagemagick', fps=10)
plt.show()